In [1]:
import gym
import gym_IRL
import random
import numpy as np
import tensorflow as tf
import time

from collections import deque
from collections import namedtuple
from matplotlib import pyplot as plt

seed = 0
np.random.seed(seed)
random.seed(seed)

env = gym.make('IRL-v0')
env.seed(seed)
policy_x = 0
policy_y = 1
P = np.zeros((11, 11, 2))
for s in range(1, 10):
    P[s, s - 1, 0] = 1
    P[s, s + 1, 1] = 1
P[0, 0, 0] = 1
P[0, 1, 1] = 1
P[10, 9, 0] = 1
P[10, 10, 1] = 1

Step = namedtuple('Step','cur_state action next_state reward done encode')
trajs = []
rand_trajs = []

for i in range(1000):
    obs = env.reset()
    done = False
    total_reward = 0
    total_loss = 0
    episode = []
    encode = 0
    #encode = random.randint(0, 1)
    for t in range(15):
        action = encode
        next_obs, reward, done, info = env.step(action)
        episode.append(Step(cur_state=obs, action=action, next_state=next_obs, 
                            reward=reward, done=done, encode=encode)) 
        obs = next_obs
        if done:
            break
    trajs.append(episode)
    
for i in range(1000):
    obs = env.reset()
    done = False
    total_reward = 0
    total_loss = 0
    episode = []
    encode = 0.5
    for t in range(15):
        action = random.randint(0, 1)
        next_obs, reward, done, info = env.step(action)
        episode.append(Step(cur_state=obs, action=action, next_state=next_obs, 
                            reward=reward, done=done, encode=encode)) 
        obs = next_obs
        if done:
            break
    rand_trajs.append(episode)    

In [2]:
from guided_cost_learning_latent import GCLAgent
Agent = GCLAgent(11, 2)
Agent.add_experience(trajs, rand_trajs)

In [3]:
for i in range(1000):
    reward_sum, all_rewards = Agent.train_model()
print(all_rewards)

[[-72.052185    4.207413 ]
 [-72.359146    4.2253404]
 [-72.35937     4.225353 ]
 [-72.35925     4.2253466]
 [-72.3593      4.225349 ]
 [-72.359146    4.22534  ]
 [-72.35929     4.2253475]
 [-72.35922     4.225344 ]
 [-72.3591      4.2253375]
 [-72.35814     4.225281 ]
 [-71.973076    4.2027955]]
